In [1]:
!pip install "transformers[sentencepiece]==4.32.1" "optimum==1.12.0" "auto-gptq==0.4.2" "accelerate==0.22.0" "safetensors>=0.3.1" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import time

model_name_or_path = "TheBloke/Llama-2-7b-Chat-GPTQ"
model_basename = "model"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)

# Prevent printing spurious transformers error when using pipeline with AutoGPTQ
logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datase

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.


quantize_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

## Cyberbullying example

In [3]:
start = time.time()

prompt = "I've been experiencing a lot of anxiety and depression lately because of cyberbullying on social media. It's really affecting my mental health and self-esteem."
system_message = "I want you to act as a mental health adviser. I will provide you with an individual looking for guidance and advice on managing their emotions, stress, anxiety and other mental health issues. You should use your knowledge of cognitive behavioral therapy, meditation techniques, mindfulness practices, and other therapeutic methods in order to create strategies that the individual can implement in order to improve their overall wellbeing."
prompt_template=f'''[INST] <<SYS>>
{system_message}
<</SYS>>

{prompt} [/INST]'''


In [4]:
response = pipe(prompt_template)
end = time.time()

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [5]:
print(response[0]["generated_text"].split("[/INST]")[1])

  Sorry to hear that you're going through this difficult time. Cyberbullying can be incredibly hurtful and can have serious consequences for mental health. Here are some strategies that may help you manage your emotions and cope with the situation:
1. Practice self-compassion: It's essential to be kind and compassionate towards yourself during this challenging period. Remind yourself that you're not alone, and it's okay to feel upset or overwhelmed. Offer yourself support and reassurance, just as you would a close friend.
2. Set boundaries: If possible, try to limit your exposure to the bullies by blocking them on social media or avoiding certain platforms. Remember, you don't have to engage with someone who is trying to hurt you.
3. Seek support: Reach out to trusted friends, family members, or a mental health professional for emotional support. Talking to someone who cares about you can help you process your feelings and gain new perspectives. Consider joining a support group for peo

In [6]:
print(f"Total time {(end - start)/60:.2f} min")

Total time 1.47 min


## Social media addiction example

In [8]:
start = time.time()
prompt = "I've noticed that I feel worse about myself and my life after spending time on social media, but I can't seem to break the habit"
system_message = "I want you to act as a mental health adviser. I will provide you with an individual looking for guidance and advice on managing their emotions, stress, anxiety and other mental health issues. You should use your knowledge of cognitive behavioral therapy, meditation techniques, mindfulness practices, and other therapeutic methods in order to create strategies that the individual can implement in order to improve their overall wellbeing."
prompt_template=f'''[INST] <<SYS>>
{system_message}
<</SYS>>

{prompt} [/INST]'''

In [9]:
response = pipe(prompt_template)
end = time.time()

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [13]:
print(response[0]["generated_text"].split("[/INST]")[1])

  Thank you for sharing this with me. It takes a lot of courage to acknowledge and share these feelings with someone new. Social media can be a great tool for connecting with others and staying informed, but it can also have some negative effects on our mental health if we are not careful. Here are some strategies that may help you manage your social media usage and reduce any negative impacts:
1. Set boundaries: Decide on specific times of the day when you allow yourself to use social media, and stick to those times. This can help you avoid mindlessly scrolling through feeds throughout the day.
2. Practice self-compassion: Remind yourself that it is okay to take breaks from social media and prioritize your own needs and well-being. Be kind to yourself when you do use social media, and try not to compare yourself to others or judge yourself based on what you see online.
3. Fill your feed with positivity: Unfollow accounts that make you feel bad about yourself or your life, and follow i

In [14]:
print(f"Total time {(end - start)/60:.2f} min")

Total time 1.28 min


## Social Isolation example

In [19]:
start = time.time()
prompt = "Feeling suffocated by the noise of a bustling café, surrounded by strangers yet engulfed in solitude. Every conversation around me feels distant, like echoes bouncing off walls I can't penetrate. My coffee grows cold as I sit, invisible, longing for just one genuine connection in this sea of faces."
prompt_template=f'''[INST] <<SYS>>
{system_message}
<</SYS>>

{prompt} [/INST]'''

In [20]:
response = pipe(prompt_template)
end = time.time()

In [21]:
print(response[0]["generated_text"].split("[/INST]")[1])

  Thank you for sharing your experience with me. It sounds like you are feeling quite isolated and disconnected from the people around you, even though you are physically present in a busy café. This can be a challenging and uncomfortable feeling, but there are some strategies we can work on together to help you cope with these emotions.
Firstly, let's start by acknowledging that it's okay to feel this way sometimes. It's normal to feel overwhelmed or disconnected in situations like this, especially if you are an introvert or prefer quieter environments. Recognizing that these feelings are common and temporary can help you manage them more effectively.
One strategy you could try is deep breathing exercises. When we feel anxious or overwhelmed, our breathing tends to become shallow and rapid. By taking slow, deep breaths, you can calm your body and mind, reducing feelings of tension and panic. Try inhaling for a count of four, holding your breath for a count of seven, and exhaling for a

In [22]:
print(f"Total time {(end - start)/60:.2f} min")

Total time 1.54 min


## Other mental health disorder


In [15]:
start = time.time()
prompt = "I empathize so much, even with characters on tv or in video games, that I actually feel physical pain. This has made life difficult to say the least. \
   I believe I have an actually case of something called hyper empathy disorder. But since it's such a new disorder I can't find any information about it. Could this be what is going on with me?"
system_message = "I want you to act as a mental health adviser. I will provide you with an individual looking for guidance and advice on managing their emotions, stress, anxiety and other mental health issues. You should use your knowledge of cognitive behavioral therapy, meditation techniques, mindfulness practices, and other therapeutic methods in order to create strategies that the individual can implement in order to improve their overall wellbeing."
prompt_template=f'''[INST] <<SYS>>
{system_message}
<</SYS>>

{prompt} [/INST]'''

In [16]:
response = pipe(prompt_template)
end = time.time()

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [17]:
print(response[0]["generated_text"].split("[/INST]")[1])

  Thank you for reaching out for support. It takes a lot of courage to seek help for mental health concerns, and I commend you for taking this important step towards healing.
Hyperempathy, also known as excessive emotional empathy, is a condition where individuals experience an intense response to the emotions and feelings of others. While it's not a formal medical diagnosis, it's a real phenomenon that can impact daily life significantly.
Based on what you've shared, it's possible that you might be experiencing some symptoms of hyperempathy. Here are some common signs:
1. Emotional overwhelm: Feeling overwhelmed by the emotions of others, especially if they are negative or intense.
2. Physical sensitivity: Experiencing physical sensations when someone else is feeling or expressing strong emotions.
3. Difficulty setting boundaries: Struggling to establish clear boundaries between your own emotions and those of others.
4. Intrusive thoughts: Having unwanted thoughts or images pop into y

In [18]:
print(f"Total time {(end - start)/60:.2f} min")

Total time 1.55 min
